In [65]:
# Load env variables and create client
from dotenv import load_dotenv
from anthropic import Anthropic

load_dotenv()

client = Anthropic()
model = "claude-3-7-sonnet-latest"

In [66]:
# Helper functions
from anthropic.types import Message


def add_user_message(messages, message):
    user_message = {
        "role": "user",
        "content": message.content if isinstance(message, Message) else message,
    }
    messages.append(user_message)


def add_assistant_message(messages, message):
    assistant_message = {
        "role": "assistant",
        "content": message.content if isinstance(message, Message) else message,
    }
    messages.append(assistant_message)


def chat(messages, system=None, temperature=1.0, stop_sequences=[], tools=None):
    params = {
        "model": model,
        "max_tokens": 1000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    if tools:
        params["tools"] = tools

    if system:
        params["system"] = system

    message = client.messages.create(**params)
    return message


def text_from_message(message):
    return "\n".join(
        [block.text for block in message.content if block.type == "text"]
    )

In [67]:
# Tools and Schemas

from datetime import datetime, timedelta
import subprocess
import json


def add_duration_to_datetime(
    datetime_str, duration=0, unit="days", input_format="%Y-%m-%d"
):
    date = datetime.strptime(datetime_str, input_format)

    if unit == "seconds":
        new_date = date + timedelta(seconds=duration)
    elif unit == "minutes":
        new_date = date + timedelta(minutes=duration)
    elif unit == "hours":
        new_date = date + timedelta(hours=duration)
    elif unit == "days":
        new_date = date + timedelta(days=duration)
    elif unit == "weeks":
        new_date = date + timedelta(weeks=duration)
    elif unit == "months":
        month = date.month + duration
        year = date.year + month // 12
        month = month % 12
        if month == 0:
            month = 12
            year -= 1
        day = min(
            date.day,
            [
                31,
                29
                if year % 4 == 0 and (year % 100 != 0 or year % 400 == 0)
                else 28,
                31,
                30,
                31,
                30,
                31,
                31,
                30,
                31,
                30,
                31,
            ][month - 1],
        )
        new_date = date.replace(year=year, month=month, day=day)
    elif unit == "years":
        new_date = date.replace(year=date.year + duration)
    else:
        raise ValueError(f"Unsupported time unit: {unit}")

    return new_date.strftime("%A, %B %d, %Y %I:%M:%S %p")


def set_reminder(content, timestamp):
    print(
        f"----\nSetting the following reminder for {timestamp}:\n{content}\n----"
    )

def kind_pods(namespace="default"):
    """
    Fetch all pods from KIND cluster using kubectl
    
    Args:
        namespace (str): Kubernetes namespace. Use "all" for all namespaces
        
    Returns:
        list: List of dictionaries containing pod information
    """
    try:
        print(f"Fetching pods in namespace: {namespace}")
        
        # Build kubectl command
        if namespace == "all":
            cmd = ["kubectl", "get", "pods", "-A", "-o", "json"]
        else:
            cmd = ["kubectl", "get", "pods", "-n", namespace, "-o", "json"]
        
        # Execute kubectl command
        result = subprocess.run(cmd, capture_output=True, text=True, check=True)
        
        # Parse JSON output
        pods_data = json.loads(result.stdout)
        
        # Extract simple pod information
        pods = []
        for pod in pods_data["items"]:
            pod_info = {
                "name": pod["metadata"]["name"],
                "namespace": pod["metadata"]["namespace"],
                "status": pod["status"]["phase"],
                "ready": "0/0",  # Default
                "restarts": 0,
                "age": pod["metadata"]["creationTimestamp"],
                "node": pod["spec"].get("nodeName", "Not scheduled")
            }
            
            # Calculate ready containers and restarts
            if "containerStatuses" in pod["status"]:
                ready_containers = sum(1 for c in pod["status"]["containerStatuses"] if c["ready"])
                total_containers = len(pod["status"]["containerStatuses"])
                total_restarts = sum(c["restartCount"] for c in pod["status"]["containerStatuses"])
                
                pod_info["ready"] = f"{ready_containers}/{total_containers}"
                pod_info["restarts"] = total_restarts
            
            pods.append(pod_info)
        
        print(f"Found {len(pods)} pods")
        return pods
    except Exception as e:
        print(f"Error fetching pods: {e}")
        return []

add_duration_to_datetime_schema = {
    "name": "add_duration_to_datetime",
    "description": "Adds a specified duration to a datetime string and returns the resulting datetime in a detailed format. This tool converts an input datetime string to a Python datetime object, adds the specified duration in the requested unit, and returns a formatted string of the resulting datetime. It handles various time units including seconds, minutes, hours, days, weeks, months, and years, with special handling for month and year calculations to account for varying month lengths and leap years. The output is always returned in a detailed format that includes the day of the week, month name, day, year, and time with AM/PM indicator (e.g., 'Thursday, April 03, 2025 10:30:00 AM').",
    "input_schema": {
        "type": "object",
        "properties": {
            "datetime_str": {
                "type": "string",
                "description": "The input datetime string to which the duration will be added. This should be formatted according to the input_format parameter.",
            },
            "duration": {
                "type": "number",
                "description": "The amount of time to add to the datetime. Can be positive (for future dates) or negative (for past dates). Defaults to 0.",
            },
            "unit": {
                "type": "string",
                "description": "The unit of time for the duration. Must be one of: 'seconds', 'minutes', 'hours', 'days', 'weeks', 'months', or 'years'. Defaults to 'days'.",
            },
            "input_format": {
                "type": "string",
                "description": "The format string for parsing the input datetime_str, using Python's strptime format codes. For example, '%Y-%m-%d' for ISO format dates like '2025-04-03'. Defaults to '%Y-%m-%d'.",
            },
        },
        "required": ["datetime_str"],
    },
}

set_reminder_schema = {
    "name": "set_reminder",
    "description": "Creates a timed reminder that will notify the user at the specified time with the provided content. This tool schedules a notification to be delivered to the user at the exact timestamp provided. It should be used when a user wants to be reminded about something specific at a future point in time. The reminder system will store the content and timestamp, then trigger a notification through the user's preferred notification channels (mobile alerts, email, etc.) when the specified time arrives. Reminders are persisted even if the application is closed or the device is restarted. Users can rely on this function for important time-sensitive notifications such as meetings, tasks, medication schedules, or any other time-bound activities.",
    "input_schema": {
        "type": "object",
        "properties": {
            "content": {
                "type": "string",
                "description": "The message text that will be displayed in the reminder notification. This should contain the specific information the user wants to be reminded about, such as 'Take medication', 'Join video call with team', or 'Pay utility bills'.",
            },
            "timestamp": {
                "type": "string",
                "description": "The exact date and time when the reminder should be triggered, formatted as an ISO 8601 timestamp (YYYY-MM-DDTHH:MM:SS) or a Unix timestamp. The system handles all timezone processing internally, ensuring reminders are triggered at the correct time regardless of where the user is located. Users can simply specify the desired time without worrying about timezone configurations.",
            },
        },
        "required": ["content", "timestamp"],
    },
}

batch_tool_schema = {
    "name": "batch_tool",
    "description": "Invoke multiple other tool calls simultaneously",
    "input_schema": {
        "type": "object",
        "properties": {
            "invocations": {
                "type": "array",
                "description": "The tool calls to invoke",
                "items": {
                    "type": "object",
                    "properties": {
                        "name": {
                            "type": "string",
                            "description": "The name of the tool to invoke",
                        },
                        "arguments": {
                            "type": "string",
                            "description": "The arguments to the tool, encoded as a JSON string",
                        },
                    },
                    "required": ["name", "arguments"],
                },
            }
        },
        "required": ["invocations"],
    },
}

kind_pods_tool = {
    "name": "kind_pods",
    "description": "Get list of pods from Kubernetes cluster, showing running, pending, and failed pods",
    "input_schema": {
        "type": "object",
        "properties": {
            "namespace": {
                "type": "string",
                "description": "Kubernetes namespace ('default', 'all', or specific namespace name)",
                "default": "default"
            }
        },
        "required": []
    }
}



In [68]:
# get_current_datetime tool function
from anthropic.types import ToolParam


def get_current_datetime(date_format="%Y-%m-%d %H:%M:%S"):
    if not date_format:
        raise ValueError("date_format cannot be empty")
    return datetime.now().strftime(date_format)


get_current_datetime_schema = ToolParam(
    {
        "name": "get_current_datetime",
        "description": "Returns the current date and time formatted according to the specified format string. This tool provides the current system time formatted as a string. Use this tool when you need to know the current date and time, such as for timestamping records, calculating time differences, or displaying the current time to users. The default format returns the date and time in ISO-like format (YYYY-MM-DD HH:MM:SS).",
        "input_schema": {
            "type": "object",
            "properties": {
                "date_format": {
                    "type": "string",
                    "description": "A string specifying the format of the returned datetime. Uses Python's strftime format codes. For example, '%Y-%m-%d' returns just the date in YYYY-MM-DD format, '%H:%M:%S' returns just the time in HH:MM:SS format, '%B %d, %Y' returns a date like 'May 07, 2025'. The default is '%Y-%m-%d %H:%M:%S' which returns a complete timestamp like '2025-05-07 14:32:15'.",
                    "default": "%Y-%m-%d %H:%M:%S",
                }
            },
            "required": [],
        },
    }
)

In [69]:
import json


def run_tool(tool_name, tool_input):
    if tool_name == "get_current_datetime":
        return get_current_datetime(**tool_input)
    elif tool_name == "add_duration_to_datetime":
        return add_duration_to_datetime(**tool_input)
    elif tool_name == "set_reminder":
        return set_reminder(**tool_input)
    elif tool_name == "kind_pods":
        return kind_pods(**tool_input)
    else:
        raise ValueError(f"Unknown tool: {tool_name}")


def run_tools(message):
    tool_requests = [
        block for block in message.content if block.type == "tool_use"
    ]
    tool_result_blocks = []

    for tool_request in tool_requests:
        try:
            tool_output = run_tool(tool_request.name, tool_request.input)
            tool_result_block = {
                "type": "tool_result",
                "tool_use_id": tool_request.id,
                "content": json.dumps(tool_output),
                "is_error": False,
            }
        except Exception as e:
            tool_result_block = {
                "type": "tool_result",
                "tool_use_id": tool_request.id,
                "content": f"Error: {e}",
                "is_error": True,
            }

        tool_result_blocks.append(tool_result_block)

    return tool_result_blocks

In [70]:
def run_conversation(messages):
    while True:
        response = chat(messages, tools=[get_current_datetime_schema, add_duration_to_datetime_schema, set_reminder_schema, batch_tool_schema, kind_pods_tool])

        add_assistant_message(messages, response)
        print(text_from_message(response))

        if response.stop_reason != "tool_use":
            break

        tool_results = run_tools(response)
        add_user_message(messages, tool_results)

    return messages

In [71]:
messages = []
add_user_message(
    messages,
    """Please analyze my Kubernetes cluster health by getting all pods across all namespaces. 

I need a detailed analysis that includes:
1. **Overall cluster summary** - total pods, health distribution
2. **Critical issues first** - any failed, pending, or crashing pods
3. **High restart alerts** - pods with >5 restarts (potential issues)
4. **Namespace breakdown** - organized by namespace for clarity
5. **Actionable recommendations** - what needs immediate attention

Format the output as a structured report with:
- Executive summary at the top
- Problem pods section (failed/pending/high restarts)
- Healthy pods summary by namespace
- Next steps and recommendations

Focus on operational insights rather than just raw data."""
)

run_conversation(messages)

I'll analyze your Kubernetes cluster health by checking the pod status across all namespaces and provide a structured report with the requested information.
Fetching pods in namespace: all
Found 32 pods
# Kubernetes Cluster Health Analysis Report

## Executive Summary
- **Total Pods**: 30
- **Health Status**: 29 running successfully (96.7%), 1 problematic pod (3.3%)
- **Namespaces**: 3 (kube-system, kubeflow, local-path-storage)
- **Critical Issues**: 1 pod with excessive restarts (proxy-agent in kubeflow namespace)
- **Overall Assessment**: Cluster is mostly healthy with one pod requiring immediate attention

## Problem Pods - Critical Issues

### Failed/Pending Pods
No completely failed or pending pods detected.

### Pods with Excessive Restarts (>5)
1. **proxy-agent-7d977cb599-786wg** (kubeflow namespace)
   - **Status**: Running but not ready (0/1)
   - **Restarts**: 857 (CRITICAL)
   - **Age**: Since 2025-08-15
   - **Node**: bitcoin-kubeflow-worker
   - **Issue**: Pod is continuo

[{'role': 'user',
  'content': 'Please analyze my Kubernetes cluster health by getting all pods across all namespaces. \n\nI need a detailed analysis that includes:\n1. **Overall cluster summary** - total pods, health distribution\n2. **Critical issues first** - any failed, pending, or crashing pods\n3. **High restart alerts** - pods with >5 restarts (potential issues)\n4. **Namespace breakdown** - organized by namespace for clarity\n5. **Actionable recommendations** - what needs immediate attention\n\nFormat the output as a structured report with:\n- Executive summary at the top\n- Problem pods section (failed/pending/high restarts)\n- Healthy pods summary by namespace\n- Next steps and recommendations\n\nFocus on operational insights rather than just raw data.'},
 {'role': 'assistant',
  'content': [TextBlock(citations=None, text="I'll analyze your Kubernetes cluster health by checking the pod status across all namespaces and provide a structured report with the requested information